# Predicting Match Outcomes

In [1]:
#SQL Imports
import mysql.connector
#Pandas imports
import pandas as pd
import datetime

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor()

In [3]:
query = pd.read_sql_query('''
    SELECT * 
    FROM TeamMatchStats
''', mydb)

In [4]:
match_stat_df = pd.DataFrame(query)

In [5]:
match_stat_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,prop_1,prop_2,hooker,sr_1,sr_2,lock_1,int_1,int_2,int_3,int_4
0,1,1,14,1,1,34,5,4,3,0.8000,...,404,396,391,407,406,483,1626,402,397,219
1,2,1,1,14,0,12,2,1,1,0.5000,...,11,500,499,2,10,269,3,400,9,1624
2,3,2,8,6,1,19,3,3,0,1.0000,...,192,503,201,196,188,197,190,429,194,505
3,4,2,6,8,0,18,3,2,1,0.6667,...,510,509,153,511,155,512,516,309,513,327
4,5,3,10,4,1,20,4,1,1,0.2500,...,268,272,265,264,375,266,260,276,267,261


# Prediction Algorithm

Things to try:
1. Want to create scoring function (sklearn.metrics import f1_score, make_scorer, classification_report)

2. Random Forest
3. Neural Network (10-3-2)

Features to try:
- home field advantage
- form in last X matches
- Points differential last X matches
- Points scored last X matches
- Points allowed last X matches
- Running metre differential last X matches
- Tackle Percentages
- Factor in the strength of opposition


In [ ]:
for match in list(match_stat_df['match_id'].unique()):
    match_df = match_stat_df[match_stat_df['match_id'] == match]
    #print(match_df)
    for team in list(match_df['team_id']):
        team_data = match_df[match_df['team_id'] == team]
        opp_data = match_df[match_df['team_id'] != team]
#         opp_points = opp_data['points'][1]
#         point_differential = team_data['points'][0] - opp_points
#         opp_tries = opp_data['tries']
#         opp_runs = opp_data['runs']
#         opp_run_metres = opp_data['run_metres']
#         opp_metres_per_run = opp_data['metres_per_run']
#         opp_kick_return_metres = opp_data['kick_return_metres']
#         opp_post_contact_metres = opp_data['post_contact_metres']
#         opp_line_breaks = opp_data['line_breaks']
#         opp_tackles = opp_data['tackles']
#         opp_missed_tackles = opp_data['missed_tackles'] + opp_data['ineffective_tackles']
#         opp_kicks = opp_data['kicks']
#         opp_kicking_metres = opp_data['metres_per_kick']
        #print(opp_data['points'].dtype)
        #points = team_data['points']
        #print(points)
        #opp_points = opp_data['points']
        #point_diff = points - opp_points
        point_diff = team_data['points'].iloc[0] - opp_data['points'].iloc[0]
        run_metre_diff = team_data['run_metres'].iloc[0] - opp_data['run_metres'].iloc[0]
        opp_missed_tackles = opp_data['missed_tackles'].iloc[0] + opp_data['ineffective_tackles'].iloc[0]
        metres_per_run_diff = float(team_data['metres_per_run'].iloc[0]) - float(opp_data['metres_per_run'].iloc[0])
        
        print(metres_per_run_diff)

1.3541000000000007
-1.3541000000000007
-1.8365
1.8365
-0.3675999999999995
0.3675999999999995
0.5433000000000003
-0.5433000000000003
9.999999999976694e-05
-9.999999999976694e-05
-1.080400000000001
1.080400000000001
1.5291999999999994
-1.5291999999999994
-0.5739000000000001
0.5739000000000001
-0.6962999999999999
0.6962999999999999
3.321300000000001
-3.321300000000001
2.5736
-2.5736
-0.03480000000000061
0.03480000000000061
-0.2812999999999999
0.2812999999999999
0.35770000000000035
-0.35770000000000035
0.8363000000000005
-0.8363000000000005
-1.0067000000000004
1.0067000000000004
-0.8142000000000005
0.8142000000000005
-0.9245000000000001
0.9245000000000001
-0.46169999999999867
0.46169999999999867
-0.28309999999999924
0.28309999999999924
1.2424999999999997
-1.2424999999999997
-0.9049999999999994
0.9049999999999994
-0.7059999999999995
0.7059999999999995
1.6693000000000016
-1.6693000000000016
-0.7157999999999998
0.7157999999999998
-0.5607000000000006
0.5607000000000006
-0.43240000000000123
0.4

In [ ]:
mycursor.close()